In [1]:
from pyspark.sql import SparkSession
import logging
import os

# Initialize logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s:%(funcName)s:%(levelname)s:%(message)s')
logger = logging.getLogger("spark_structured_streaming")

In [2]:
def initialize_spark_session(app_name, access_key, secret_key, minio_endpoint):
    """
    Initialize the Spark Session with MinIO configurations, Delta Lake support, and additional packages.
    
    :param app_name: Name of the Spark application.
    :param access_key: Access key for MinIO.
    :param secret_key: Secret key for MinIO.
    :param minio_endpoint: MinIO server endpoint (including port).
    :return: Spark session object or None if there's an error.
    """
    # Set the desired JAVA_HOME
    os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_321.jdk/Contents/Home'
    os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:" + os.environ['PATH']
    
    # Add the Scala path to the PATH environment variable
    os.environ['PATH'] = "/opt/homebrew/opt/scala@2.13/bin:" + os.environ['PATH']
    
    
    try:
        # #.master("spark://spark_master:7077") \
        spark = SparkSession \
                .builder \
                .appName(app_name) \
                .config("spark.hadoop.fs.s3a.access.key", access_key) \
                .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
                .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
                .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
                .config("spark.hadoop.fs.s3a.path.style.access", "true") \
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                .config("spark.hadoop.fs.s3a.aws.credentials.provider", 
                        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                .config("spark.jars.packages", 
                        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
                        "org.apache.kafka:kafka-clients:3.3.0,"
                        "org.apache.hadoop:hadoop-aws:3.2.0,"
                        "com.amazonaws:aws-java-sdk-s3:1.11.375,"
                        "org.postgresql:postgresql:42.2.18,"
                        "org.apache.commons:commons-pool2:2.8.0,"
                        "io.delta:delta-core_2.12:2.1.0") \
                .getOrCreate()

        spark.sparkContext.setLogLevel("ERROR")
        logger.info('Spark session initialized successfully with MinIO, Delta Lake, and additional packages')
        return spark

    except Exception as e:
        logger.error(f"Spark session initialization failed. Error: {e}")
        return None

In [3]:

# Replace with your actual MinIO credentials and endpoint
app_name = "DeltaLakeMinIOExplorer"
minio_endpoint = "http://minio:9000"
access_key = "H3jy2qkZ6ZFASS7slRJe"
secret_key = "uFdq1hFXiee8pT2hG52gguyI0Iqkkyro07dDjopB"
brokers = "localhost:29092,localhost:29093,localhost:29094"


In [4]:
spark = initialize_spark_session(app_name, access_key, secret_key, minio_endpoint)
spark

24/08/21 15:53:15 WARN Utils: Your hostname, Hamzas-MacBook-Pro-8665.local resolves to a loopback address: 127.0.0.1; using 10.50.49.133 instead (on interface en14)
24/08/21 15:53:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/Homebrew/Caskroom/miniforge/base/envs/data_pipeline/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/hamzaharunamohammed/.ivy2/cache
The jars for the packages stored in: /Users/hamzaharunamohammed/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-s3 added as a dependency
org.postgresql#postgresql added as a dependency
org.apache.commons#commons-pool2 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5cb2747a-f9e6-4fca-80e1-143ac997a016;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found org.xerial.snappy#snappy-java;1.1

24/08/21 15:53:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/21 15:53:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


2024-08-21 15:53:24,620:initialize_spark_session:INFO:Spark session initialized successfully with MinIO, Delta Lake, and additional packages


In [5]:
delta_table_path = "s3a://datalake/processed_data/deltatable/"  # MinIO bucket path for output data
checkpoint_location = "s3a://datalake/checkpoints/spark_metadata"  # Correct checkpoint location path


# Load the Delta table
delta_df = spark.read.format("delta").load(delta_table_path)

# Show the first few records to explore the data
#delta_df.show()


Py4JJavaError: An error occurred while calling o52.load.
: org.apache.hadoop.fs.s3a.AWSClientIOException: doesBucketExist on datalake: com.amazonaws.SdkClientException: Unable to execute HTTP request: minio: Unable to execute HTTP request: minio
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:189)
	at org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:111)
	at org.apache.hadoop.fs.s3a.Invoker.lambda$retry$3(Invoker.java:265)
	at org.apache.hadoop.fs.s3a.Invoker.retryUntranslated(Invoker.java:322)
	at org.apache.hadoop.fs.s3a.Invoker.retry(Invoker.java:261)
	at org.apache.hadoop.fs.s3a.Invoker.retry(Invoker.java:236)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.verifyBucketExists(S3AFileSystem.java:375)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:311)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getI